# Resultados de la competición de clasificación con modelos de Regresión Logística

**Tarea:** [California Housing](https://scikit-learn.org/dev/datasets/real_world.html#california-housing-dataset)

## 0a. Librerías

In [5]:
import sys
import warnings; warnings.filterwarnings("ignore"); 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## 0b. Funciones y variables dependientes de la tarea

In [6]:
# MODIFICAR PARA ADAPTAR A LA TAREA Y CONDICIONES DEL EXAMEN

from sklearn.datasets import fetch_california_housing as load_corpus

RANDOM_SEED = 22 # Semilla para el barajado aleatorio de datos y para el constructor de LogisticRegression()
TEST_SIZE = 0.2 # Porcentaje de datos de test
NOTA_MAX = 0.75 # puntuación máxima del examen

## 1. Clase `LRSystem`

Esta clase representa un sistema de regresión logística participante en la competición.

In [7]:
class LRSystem:

    # Carga estática del corpus y partición
    corp = load_corpus()
    X = corp.data.astype(np.float16) # muestras
    y = corp.target.astype(np.uint)  # etiquetas de clase
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=TEST_SIZE, 
                                                        random_state=RANDOM_SEED)
    
    def __init__(self,  owner, tol=None, C=None, max_iter=None, obs=""):
        self.owner = owner
        self.tol = tol
        self.C = C
        self.max_iter = max_iter
        self.obs = obs
        self.err_train = None
        self.err_test = None
        self.nota = None
        self.eval() # Evaluamos sistema en su creación
    
    def eval(self):
        sys.stderr.write(f"Assessing LR system from {self.owner}... \n")
        clf = LogisticRegression(random_state=RANDOM_SEED, 
                                 C=self.C, 
                                 tol=self.tol, 
                                 max_iter=self.max_iter).fit(self.X_train, self.y_train)
        self.err_train = (1 - accuracy_score(self.y_train, clf.predict(self.X_train))) * 100
        self.err_test = (1 - accuracy_score(self.y_test, clf.predict(self.X_test))) * 100

## 2. Sistemas participantes

Se definen los sistemas participantes, como instancias de `LRSystem`, usando los valores de los hiperparámetros `tol`, `C`, y `max_iter` reportados en la celda de texto final del cuaderno Jupyter entregado en el examen. Notar que el constructor de `LRSystem` invoca el método `eval()`, el cual que se encarga de entrenar el clasificador con dichos hiperparámetros y evaluar sus prestaciones en los conjuntos de train y de test.

Se incluye en la competición, además, el sistema base proporcionado (*baseline*), cuya tasa de error en test sirve como referencia para calcular la nota de la práctica.

In [8]:
systems = []
systems.append(LRSystem("**BASELINE**", C=1, tol=0.01, max_iter=10))

systems.append(LRSystem("RdA", C=10, tol=1e-4, max_iter=200, 
                        obs="Había usado un Standard Scaler para preprocesar datos. Eso no está permitido."))
systems.append(LRSystem("VE", C=1, tol=1e-3, max_iter=200))
systems.append(LRSystem("NL", C=0.1, tol=1e-4, max_iter=500))
systems.append(LRSystem("VM", C=0.001, tol=1e-4, max_iter=100))
systems.append(LRSystem("JM", C=1000, tol=1e-3, max_iter=5000))
systems.append(LRSystem("UM", C=10, tol=1e-2, max_iter=10000))

Assessing LR system from **BASELINE**... 
Assessing LR system from RdA... 
Assessing LR system from VE... 
Assessing LR system from NL... 
Assessing LR system from VM... 
Assessing LR system from JM... 
Assessing LR system from UM... 


# 3. Cálculo de la nota de la práctica

La nota $N$ del participante, acotada superiormente por $N_\text{max}$, se calcula en función del cociente de la diferencia entre la tasa de error del sistema del participante $\hat{\varepsilon}_{\text{system}}$ y la mínima tasa de error $\hat{\varepsilon}_{best}$ obtenida por el sistema ganador de la competición, y la diferencia entre la tasa de error de referencia del sistema base $\hat{\varepsilon}_{\text{baseline}}$ y la mínima tasa de error $\hat{\varepsilon}_{best}$. 

$$ N = \max \left( \; 0 \, , \; \left( 1 - \frac{|\hat{\varepsilon}_{\text{system}} - \hat{\varepsilon}_{best}|}{|\hat{\varepsilon}_{baseline} - \hat{\varepsilon}_{best}|} \right) \cdot N_\text{max} \; \right) $$

En otras palabras, la nota $N$ es la proporción de la mejora obtenida sobre el sistema base respecto a la máxima mejora posible (obtenida por el sistema ganador).

Nota que, en el fatal caso que $\hat{\varepsilon}_{\text{system}} \gt \hat{\varepsilon}_{baseline}$, y a fin de prevenir la asignación de una nota negativa, se escogerá siempre el máximo entre $0$ y el mencionado cociente:

De este modo, el ganador de la competición obtiene la máxima nota ($N = N_\text{max}$), y el resto de participantes, una nota que depende linealmente de la distancia con el ganador.

#### Ejemplo:

La nota máxima es $N_\text{max} = 0.75$, y el sistema base tiene una tasa de error $\hat{\varepsilon}_{baseline} = 10\%$. El sistema ganador obtiene $\hat{\varepsilon}_{best} = 5\%$, y el sistema que estamos evaluando obtiene $\hat{\varepsilon}_{system} = 7.5\%$:

$$ N = \max \left( \; 0 \, , \; \left( 1 - \frac{|7.5 - 5|}{|10 - 5|} \right) \cdot 0.75 \; \right) = \max ( \; 0, \; 0.5 \cdot 0.75 \; ) = 0.375$$

**Interpretación:** el sistema participante solo ha podido "capturar" un 50% de la mejora obtenida por el sistema ganador, por tanto, le corresponde el 50% de la nota máxima.



 

In [9]:
def calc_mark(bl_err, min_err, participant_err):
    return (1 - abs(participant_err - min_err) / abs(bl_err - min_err)) * NOTA_MAX

bl_err = systems[0].err_test
min_err = min([sys.err_test for sys in systems])

for syst in systems:
    syst.nota = calc_mark(bl_err, min_err, syst.err_test)

# 4. Tabla de resultados (clasificación) 

In [10]:
df = pd.DataFrame([vars(syst) for syst in systems])

pd.set_option('display.max_colwidth', None)
df = df[["owner", "C", "tol", "max_iter", "err_train", "err_test", "nota", "obs"]]
df = df.sort_values(['err_test','err_train'])
df = df.rename(columns = {"owner":"Participante", 
                          "err_test":"Error test (%)",
                          "err_train":"Error train (%)",
                          "nota":f"Nota [máx. {NOTA_MAX}]",
                          "obs":"Observaciones"})

formats = {"Error train (%)":"{:3.1f}".format, 
           "Error test (%)":"{:3.1f}".format,
           f"Nota [máx. {NOTA_MAX}]":"{:.2f}".format,
           "C":"{:g}".format,
           "tol":"{:g}".format,}
df.style.format(formats).relabel_index(list(map(lambda x:x+1, range(len(systems)))))

,Participante,C,tol,max_iter,Error train (%),Error test (%),Nota [máx. 0.75],Observaciones
1,UM,10,0.01,10000,44.0,45.0,0.75,
2,JM,1000,0.001,5000,44.3,45.3,0.74,
3,NL,0.1,0.0001,500,51.7,51.2,0.44,
4,RdA,10,0.0001,200,57.8,58.2,0.09,Había usado un Standard Scaler para preprocesar datos. Eso no está permitido.
5,VE,1,0.001,200,59.2,59.2,0.04,
6,VM,0.001,0.0001,100,59.5,59.5,0.02,
7,**BASELINE**,1,0.01,10,59.8,59.9,0.00,
